In [1]:
import pandas as pd
import papermill as pm
from datetime import datetime, time
import pickle
from utils.utils_dates import map_month_folder
import os
from pathlib import Path
import sys

In [3]:
# Parameters
nb_name = "5_hunt_invoices_out"
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
nb_verif = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/5b_ver_notebook.ipynb"
issue_year = 2025
issue_month = 6
cliente = "Generic"
alias = "TRIAL"
tarifa = "6.1 TD"
cups = "TRIAL"
path_load_parquet = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/elec_load.parquet"
path_contract_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/raw/contract_matrix_TRIAL.pkl"
path_coberturas_df = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/raw/df_coberturas_1.pkl"
path_master_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/raw/master_matrix_TRIAL.pkl"
db_elec_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec_manual.pkl"
template_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/templates/Template 2024.xlsx"


In [ ]:
from config.config_loader import Config

config = Config(config_file=config_path)

In [ ]:
from utils.logging_setup import get_logger
import time as tm

instant = tm.strftime("%Y-%m-%d_%H:%M:%S", tm.localtime())
logger = get_logger(name=f"{nb_name}")
logger.info(f"Building omie matrix on {instant}")

In [5]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

logger.info(f"Retrieved invoices from database")

2025-08-06 19:51:05,637 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2025-08-06 19:51:05,637 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")


2025-08-06 19:51:05,638 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 19:51:05,638 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")


2025-08-06 19:51:05,638 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 19:51:05,639 INFO sqlalchemy.engine.Engine SELECT * FROM facturas


2025-08-06 19:51:05,639 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 19:51:05,652 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
logger.info(f"Filtering invoices for CUPS: {cups}, issued year: {issue_year}, issued month: {issue_month}")
# We first look for issued invoices in our target month-year

df_filtered_issued = facturas_df[
                        (facturas_df['cups'] == cups) &
                        (facturas_df['issue_date'].dt.month == issue_month) &
                        (facturas_df['issue_date'].dt.year == issue_year)
                        ]

# Now we look for all the invoices that have the start and end invoice dates of the invoices issued
# on our target month-year

df_issued_compile = facturas_df[
                        (facturas_df['cups'] == cups) &
                        (facturas_df['inicio_periodo'].isin(df_filtered_issued['inicio_periodo'])) &
                        (facturas_df['fin_periodo'].isin(df_filtered_issued['fin_periodo']))
                        # eliminar el filtro de debajo, es solo para Kem One
                        # &
                        # (facturas_df['num_factura'].str.contains('ELE'))
                        ]

df_issued_compile = df_issued_compile.copy()
df_issued_compile.drop(['issue_date', 'year', 'month'], axis=1, inplace=True)

# Group by 'cups', 'inicio_periodo', 'fin_periodo' while keeping invoice references
ds = df_issued_compile.groupby(['cups', 'inicio_periodo', 'fin_periodo']).agg({
    'num_factura': lambda x: ', '.join(x.astype(str)),  # Concatenate all invoice references
    **{col: 'sum' for col in df_issued_compile.columns if col not in ['cups', 'inicio_periodo', 'fin_periodo', 'num_factura']}
})


df = pd.DataFrame(ds)
df.reset_index(drop=False, inplace=True)
# df_issued_compile
logger.info(f"Found {len(ds)} invoices for CUPS {cups} in the month {issue_month} of year {issue_year}")

,cups,inicio_periodo,fin_periodo,num_factura,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,total_bruto_ie_iva,...,energia_p3,energia_p4,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6
0,ES0021000005776632EH,2025-05-01,2025-05-31,FELEC_2500427045,1737.46,3546.6,0.0,0.0,0.0,5284.06,...,0.0,15436.0,10983.0,23239.0,0.0,0.0,0.0,237.0,217.0,148.0


In [8]:
if not df.empty:

    for index, row in df.iterrows():

        start_date = row['inicio_periodo'].date()
        end_date = row['fin_periodo'].date()
        start_date = datetime.combine(start_date, time(0,0,0))
        end_date = datetime.combine(end_date, time(23,0,0))
        start_path = start_date.strftime("%Y%m%d")
        end_path = end_date.strftime("%Y%m%d")
        invoice_refs = row['num_factura']
        invoice_dict = row.to_dict()
        month_clean = map_month_folder(issue_month)

        invoice_path_pickle = \
            config.get_path("outputs.verification_project.verif.cliente.invoice_path_pickle", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_excel = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_excel", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_pdf = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_pdf", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_nb = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_nb", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        
        for output_path in [invoice_path_pickle, output_verif_excel, output_verif_nb, output_verif_pdf]:

            if not os.path.exists(os.path.dirname(output_path)):

                os.makedirs(os.path.dirname(output_path))
                print(output_path)

        with open(invoice_path_pickle, "wb") as pickle_file:
            pickle.dump(invoice_dict, pickle_file) 

        logger.info(f"Executing verification for invoices:\n{invoice_refs}")
        pm.execute_notebook(
            nb_verif, output_verif_nb,##############
            parameters=dict(
                nb_name = Path(output_verif_nb).stem,
                config_path = config_path,
                start_date = start_date.isoformat(),
                end_date = end_date.isoformat(),
                cliente=cliente,
                alias = alias,
                tarifa = tarifa,
                cups = cups,
                path_load_parquet = path_load_parquet,
                path_coberturas_df = path_coberturas_df,
                path_master_matrix = path_master_matrix,
                path_contract_matrix = path_contract_matrix,
                db_elec_path = db_elec_path,
                template_path = template_path,
                invoice_path_pickle = invoice_path_pickle,
                output_verif_excel =  output_verif_excel,
                output_verif_pdf = output_verif_pdf
            ),
            stdout_file=sys.stdout,
            stderr_file=sys.stderr,
            log_output=True
        )
        logger.info(f"Verification notebook executed successfully for invoices: {invoice_refs}")
        logger.info(f"Verification PDF can be found at:\n {output_verif_pdf}")
        logger.info(f"Verification KPIs can be found at:\n {output_verif_nb}")

else:
    logger.info(f"No invoices found for CUPS {cups} in the month {issue_month} of year {issue_year}.")

Executing:   0%|          | 0/20 [00:00<?, ?cell/s]

2025-05-01 00:00:00
/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/outputs/verification_project/verif/Grupo Inspired/pdfs/2025/06. Junio/Verificación Fra. 20250501_20250531 KC SOTO VIÑUELAS 1.pdf
